In [ ]:
#------------------------
# Google Colab上でのみ実行
#------------------------
import time
!git clone https://github.com/konnitiha3/MOD2NN.git

import sys
sys.path.append('/content/MOD2NN')

from google.colab import drive
drive.mount('/content/drive')

In [1]:
import time
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
from cxlayers import ImageToElectricField, CxMO, AngularSpectrum, CxD2NNIntensity, D2NNMNISTDetector, ImageResizing, ImageBinarization, D2NNMNISTFilter, CxD2NNFaradayRotation, Polarizer

tf.random.set_seed(1)
tf.experimental.numpy.experimental_enable_numpy_behavior()

print("TensorFlow:", tf.__version__)
print("Python:", sys.version)

plt.rcParams['font.size'] = 18

TensorFlow: 2.8.0
Python: 3.8.11 (default, Aug 16 2021, 12:04:33) 
[Clang 12.0.0 ]


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

In [3]:
#@title デフォルトのタイトル テキスト
wavelength = 531.0e-9 #@param {type:"number"}
d = 1.0e-6 #@param {type:"number"}

In [4]:
theta = np.arcsin(wavelength/(2*d))
z = np.sqrt((100*d)**2 + (100*d)**2)/np.tan(theta)
z

0.0005135437489912859

In [8]:
shape = (40, 40)

inputs = tf.keras.Input((28, 28))
x = ImageResizing(shape)(inputs)
x = ImageBinarization(0.5, 0.0, 1.0)(x)
x = ImageToElectricField(shape)(x)
x = CxMO(shape, limitation='sin', limitation_num=2.1*np.pi/180)(x)
x = AngularSpectrum(shape, wavelength, 0.5e-3, d=d, normalization='max', method='band_limited')(x)
x = CxMO(shape, limitation='sin', limitation_num=2.1*np.pi/180)(x)
x = AngularSpectrum(shape, wavelength, 2.0e-3, d=d, normalization='max', method='band_limited')(x)
x = CxD2NNFaradayRotation(shape, normalization='minmax')(x)
x = D2NNMNISTDetector(10, activation='softmax')(x)
model = tf.keras.Model(inputs, x)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 image_resizing_1 (ImageResi  (None, 40, 40)           0         
 zing)                                                           
                                                                 
 image_binarization_1 (Image  (None, 40, 40)           0         
 Binarization)                                                   
                                                                 
 image_to_electric_field_1 (  (None, 2, 2, 40, 40)     0         
 ImageToElectricField)                                           
                                                                 
 cx_mo_2 (CxMO)              (None, 2, 2, 40, 40)      1600      
                                                           

In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [9]:
def loss(model, x, y):
  y_ = model(x)

  return loss_object(y_true=y, y_pred=y_)


l = loss(model, x_train, y_train)
print("Loss test: {}".format(l))

2022-04-03 18:52:45.701755: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-03 18:52:50.953599: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-03 18:53:53.696737: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Loss test: 2.3027076721191406
